In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition.pca import PCA
from sklearn.preprocessing import scale

In [16]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [17]:
def refine_data(arr1,arr2,arr3):
    refined_data = []
    for i in range(arr1.shape[0]):
        rt = find_nearest(arr1[i],arr3[i][0])
        refined_data.append([rt,arr2[i][arr1[i].tolist().index(rt)]])
        
    refined_data = np.array(refined_data)
    return refined_data

In [18]:
def get_metrics(arr1,arr2):
    mse = mean_squared_error(arr1,arr2)
    r2 = r2_score(arr1,arr2)
    return mse,r2

In [19]:
d1 = pd.read_csv('X_data_t3',sep = ',')
d2 = pd.read_csv('y_data',sep = ',')
d3 = pd.concat([d1,d2],axis=1)

#data = d3.loc[(d3['maxRT_ab'] < 1e6) & (d3['maxRT_ab'] > 3e4)]
data = d3.loc[(d3['maxRT_ab'] > 3e4)]
#data = d3.loc[(d3['maxRT_ab'] < 1e6)]
#d3 = shuffle(d3)

#X = np.array(d3[['maxRT_t','x_start_t','diff_start','diff_end']])
#X = np.array(d3[['maxRT_ab','maxRT_baseline','x_start_ab','x_end_ab']])
#X = np.array(d3[['maxRT_t','maxRT_ab','maxRT_baseline','x_start_ab','x_end_ab']])
X = np.array(d3[['rt','maxRT_t','maxRT_ab','maxRT_base','x_start_t','x_start_ab','diff_start','x_end_t','x_end_ab','diff_end','width']])

#X = np.array(d3[['rt','maxRT_ab','x_start_t','x_end_t','width']])


y = np.array(d3[['y_left_t','y_left_ab']])


time = np.array(pd.read_csv('time',sep = ',',header = None).dropna(axis = 'columns').round(3))
abundance = np.array(pd.read_csv('abundance',sep = ',',header = None).dropna(axis = 'columns'))
baseline = np.array(pd.read_csv('baseline',sep = ',',header = None).dropna(axis = 'columns'))
#time = time[0:420]
#abundance = abundance[0:420]
#baseline = baseline[0:420]

In [20]:
np.amax(d3['maxRT_ab']), np.amin(d3['maxRT_ab']), X.shape

(99672936.0, 22512.0, (423, 11))

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
scaler_train = MinMaxScaler(feature_range=(0,1))
#scaler_train = StandardScaler()
scaler_train.fit(X_train)
X_train =scaler_train.transform(X_train)

scaler_test = MinMaxScaler(feature_range=(0,1))
#scaler_test = StandardScaler()
scaler_test.fit(X_test)
X_test = scaler_test.transform(X_test)

In [23]:
n_estimators = 1000
max_depth = 1000
random_state = 1000
ite = 1

#mse_t,r2_t = [[]for i in range(len(n_estimators))], [[] for i in range(len(n_estimators))]
#mse_ab,r2_ab = [[]for i in range(len(n_estimators))], [[] for i in range(len(n_estimators))]


params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': 20,
                 'n_jobs': -1, 'random_state':random_state}
        
clf = RandomForestRegressor(**params)


scores = cross_val_score(clf, X_train, y_train, cv=5)
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test).round(3)
    
mse_1,r2_1 = get_metrics(y_test[:,0],y_predict[:,0])
mse_2,r2_2 = get_metrics(y_test[:,1],y_predict[:,1])
print(mse_1,r2_1,mse_2,r2_2)
    
    
        


0.0002529294117647059 -0.0052749823140381125 15299224.448273474 0.5448056130182723


In [24]:
y_test[:,0]

array([0.98 , 0.99 , 1.003, 1.013, 1.   , 0.993, 0.98 , 1.007, 0.983,
       0.987, 0.98 , 1.013, 1.003, 1.017, 0.997, 0.99 , 0.953, 1.   ,
       0.98 , 0.983, 0.97 , 0.983, 1.017, 0.997, 0.99 , 1.013, 0.993,
       0.997, 0.973, 0.98 , 0.973, 0.997, 0.97 , 0.98 , 0.993, 1.007,
       0.977, 0.983, 0.977, 0.983, 0.983, 0.98 , 0.98 , 0.983, 1.   ,
       0.967, 0.997, 1.017, 0.957, 0.98 , 0.993, 1.   , 0.987, 0.983,
       0.997, 1.01 , 0.953, 1.007, 0.987, 0.983, 0.983, 0.983, 0.983,
       0.983, 1.02 , 0.987, 0.963, 0.98 , 1.017, 0.973, 0.987, 0.963,
       1.003, 1.007, 0.997, 0.937, 0.987, 0.987, 0.997, 0.993, 0.98 ,
       0.977, 1.013, 0.977, 0.987])

In [25]:
y_predict[:,0]

array([0.987, 1.002, 0.998, 1.004, 1.004, 0.994, 0.989, 0.989, 0.993,
       0.987, 0.991, 0.994, 0.988, 0.987, 0.991, 0.996, 0.99 , 0.987,
       0.99 , 0.992, 0.998, 0.987, 1.002, 1.001, 0.989, 1.003, 0.994,
       0.992, 0.991, 0.992, 0.996, 0.997, 0.991, 0.991, 0.993, 1.004,
       0.989, 0.993, 0.989, 0.997, 0.991, 0.987, 1.004, 0.992, 1.001,
       0.997, 0.987, 1.012, 0.99 , 0.985, 0.993, 1.   , 0.988, 0.992,
       1.001, 0.995, 0.996, 0.994, 0.991, 0.989, 0.996, 0.993, 0.987,
       0.994, 1.029, 0.99 , 0.994, 0.987, 0.998, 0.989, 1.001, 1.   ,
       0.999, 1.   , 0.993, 0.988, 1.   , 0.988, 1.   , 1.001, 1.003,
       1.001, 0.994, 1.   , 0.994])

In [26]:
from sklearn.decomposition.pca import PCA
from sklearn.preprocessing import scale
pca = PCA()
X_reduced = pca.fit_transform(X_train)
scores = cross_val_score(clf, X_reduced, y_train, cv=5)
print(scores)
clf.fit(X_reduced,y_train)
y_predict = clf.predict(X_test).round(3)
        
        
mse_1,r2_1 = get_metrics(y_test[:,0],y_predict[:,0])
mse_2,r2_2 = get_metrics(y_test[:,1],y_predict[:,1])
        
        
print(r2_1,r2_2)
print(mse_1,mse_2)


[0.73976687 0.75249782 0.67413778 0.72655178 0.36610512]
-0.07639565062882814 -5.2099801026128905
0.00027082352941176503 208719356.22747636


In [27]:
y_predict[:,0]

array([0.993, 0.987, 0.987, 0.987, 0.987, 0.987, 0.993, 0.987, 0.987,
       0.992, 0.992, 0.987, 0.993, 0.987, 0.987, 0.987, 0.987, 0.993,
       0.992, 0.987, 0.987, 0.992, 0.987, 0.987, 0.99 , 0.987, 0.99 ,
       0.987, 0.993, 0.992, 0.987, 0.987, 0.987, 0.992, 0.987, 0.987,
       0.992, 0.987, 0.99 , 0.987, 0.993, 0.992, 0.987, 0.987, 0.987,
       0.987, 0.993, 0.987, 0.987, 0.987, 0.987, 0.987, 0.992, 0.987,
       0.987, 0.988, 0.987, 0.987, 0.987, 0.993, 0.987, 0.992, 0.993,
       0.987, 0.986, 0.987, 0.987, 0.992, 0.987, 0.993, 0.987, 0.987,
       0.987, 0.987, 0.987, 0.987, 0.987, 0.987, 0.987, 0.987, 0.987,
       0.987, 0.987, 0.987, 0.987])

In [28]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

array([ 89.6 ,  95.53,  97.73,  99.3 ,  99.96, 100.  , 100.  , 100.  ,
       100.  , 100.  , 100.  ])